In [1]:
import re
from itertools import count
from dateutil.parser import parse
from pytz import timezone
import numpy as np
import pandas as pd
from nameparser import HumanName
from fuzzywuzzy import fuzz

In [2]:
all_emails = pd.read_csv('Hillary_Emails.csv')

In [3]:
def match_emails(name_df):
    for email in emails_to_match:
        for i, row in name_df.iterrows():
            if (len(row.LastName) > 1) & ((row.LastName.lower() in email) or (fuzz.ratio(row.LastName.lower(), email.split('@')[0]) > 85)):
                if (len(row.FirstName) > 0) & (row.FirstName[0].lower() in email):
                    if email not in row['email_address']:
                        row['email_address'].append(email)
            elif (row.MetadataTo.lower() in email):
                row['email_address'].append(email)
    return name_df

In [14]:
def link_names(name_df):
    for idx, r in name_df.iterrows():
        for i, row in name_df.iterrows():
            if (len(row.LastName) > 1) & (row.LastName.lower() in r.MetadataTo.lower()) & (' ' not in r.MetadataTo.lower()):
                if (len(row.FirstName) > 0):
                    if row.FirstName.lower()[0] in r.MetadataTo.lower():
                        r['FirstName'] = row.FirstName
                        r['LastName'] = row.LastName
    return name_df

In [15]:
metadata_to = pd.DataFrame(all_emails['MetadataTo'].dropna().unique(), columns=['MetadataTo'])
metadata_to_emails = metadata_to[metadata_to['MetadataTo'].str.contains("@")]
emails_to_match = metadata_to_emails.MetadataTo.str.lower().to_list()
metadata_to_names = metadata_to[~metadata_to['MetadataTo'].str.contains("@")]
metadata_to_names = pd.DataFrame(metadata_to_names.MetadataTo.str.lower().unique(), columns=['MetadataTo'])
metadata_to_names['MetadataTo'] = metadata_to_names.MetadataTo.apply(lambda s: s.replace('.', ' '))
metadata_to_names['FirstName'] = metadata_to_names.MetadataTo.apply(lambda s: HumanName(s).first)
metadata_to_names['MiddleName'] = metadata_to_names.MetadataTo.apply(lambda s: HumanName(s).middle)
metadata_to_names['LastName'] = metadata_to_names.MetadataTo.apply(lambda s: HumanName(s).last)
metadata_to_names['email_address'] = np.empty((len(metadata_to_names), 0)).tolist()

In [16]:
metadata_to_names = match_emails(metadata_to_names)
metadata_to_names = link_names(metadata_to_names)
metadata_to_names = match_emails(metadata_to_names)

In [17]:
metadata_to_names

,MetadataTo,FirstName,MiddleName,LastName,email_address
0,h,h,,,"[abedinh@state.gov, hanleymr@state.gov, abedin..."
1,;h,;h,,,[]
2,"abedin, huma",huma,,abedin,"[abedinh@state.gov, abendinh@state.gov]"
3,"sherman, wendy r",wendy,r,sherman,[]
4,"mills, cheryl d",cheryl,d,mills,"[millscd@state.gov, cheryl.mills@]"
5,"flores, oscar",oscar,,flores,[]
6,"schwerin, daniel b",daniel,b,schwerin,[schwerindb@state.gov]
7,"nides, thomas r",thomas,r,nides,[nidestr@state.gov]
8,"sullivan, jacob j",jacob,j,sullivan,"[sullivanjj@state.gov, jake.sullivan@]"
9,jake sullivan,jake,,sullivan,"[sullivanjj@state.gov, jake.sullivan@]"
